# 0.0 IMPORTS

In [1]:
import math
import numpy    as np
import pandas   as pd
import inflection
import datetime as dt
import seaborn  as sns
import xgboost  as xgb
import random
import pickle

from scipy                import stats as ss
from boruta               import BorutaPy
from tabulate             import tabulate
from matplotlib           import pyplot as plt
from IPython.display      import Image
from IPython.display import HTML

from sklearn.metrics       import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder


## 0.1 Helper Functions

In [2]:
def cross_validation(x_training, kfold, model_name, model, verbose=False):
    # cria listas vazias para armazenar as métricas de erro de cada dobra
    mae_list = []
    mape_list = []
    rmse_list = []

    # itera através das dobras, começando pela última e indo até a primeira
    for k in reversed(range(1, kfold + 1)):
        # imprime o número da dobra atual, se verbose for True
        if verbose:
            print('\nKFold Number: {}'.format(k))

        # define as datas de início e término para a validação
        validation_start_date = x_training['date'].max() - dt.timedelta(days=k*6*7)
        validation_end_date = x_training['date'].max() - dt.timedelta(days=(k-1)*6*7)

        # filtra o conjunto de dados para obter o conjunto de treinamento e validação
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]

        # separa as variáveis de entrada e saída para o conjunto de treinamento e validação
        xtraining = training.drop(['date', 'sales'], axis=1)
        ytraining = training['sales']
        xvalidation = validation.drop(['date', 'sales'], axis=1)
        yvalidation = validation['sales']

        # treina o modelo com o conjunto de treinamento
        m = model.fit(xtraining, ytraining)

        # faz a previsão com o conjunto de validação
        yhat = m.predict(xvalidation)

        # calcula as métricas de erro e armazena em listas
        m_result = ml_error(model_name, np.expm1(yvalidation), np.expm1(yhat))
        mae_list.append(m_result['MAE'])
        mape_list.append(m_result['MAPE'])
        rmse_list.append(m_result['RMSE'])

    # calcula a média e o desvio padrão das métricas de erro e retorna em um DataFrame
    return pd.DataFrame({
        'Model Name': model_name,
        'MAE CV': np.round(np.mean(mae_list), 2).astype(str) + ' +/- ' + np.round(np.std(mae_list), 2).astype(str),
        'MAPE CV': np.round(np.mean(mape_list), 2).astype(str) + ' +/- ' + np.round(np.std(mape_list), 2).astype(str),
        'RMSE CV': np.round(np.mean(rmse_list), 2).astype(str) + ' +/- ' + np.round(np.std(rmse_list), 2).astype(str)
    }, index=[0])

def mean_percentage_error(y, yhat):
    # Calcula o erro percentual médio entre os valores reais e as previsões do modelo.
    return np.mean((y - yhat) / y) * 100



# esta função calcula os erros de desempenho (MAE, MAPE e RMSE) entre os valores reais e previstos
def ml_error( model_name, y, yhat ):
    # cálculo do erro absoluto médio (MAE)
    mae = mean_absolute_error( y, yhat )
    # cálculo do erro percentual absoluto médio (MAPE)
    mape = mean_absolute_percentage_error( y, yhat )
    # cálculo da raiz do erro quadrático médio (RMSE)
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    # armazenando os resultados em um dataframe pandas
    return pd.DataFrame( { 'Model Name': model_name,
                           'MAE': mae,
                           'MAPE': mape,
                           'RMSE': rmse }, index=[0] )


# calcula a medida de associação entre duas variáveis categóricas usando o coeficiente de Cramer's V
def cramer_v( x, y ):
    # cria uma tabela de contingência a partir das duas variáveis categóricas x e y, e transforma em uma matriz NumPy
    cm = pd.crosstab( x, y ).to_numpy()
    n = cm.sum()
    r, k = cm.shape

    # calcula o qui-quadrado para a tabela de contingência cm, e retorna o valor qui-quadrado.
    chi2 = ss.chi2_contingency( cm )[0]
    
    # calcula o valor corrigido de qui-quadrado, que leva em conta o tamanho da tabela de contingência.
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )

    # calcula o número de colunas corrigido.
    kcorr = k - (k-1)**2/(n-1)

    # calcula o número de linhas corrigido.
    rcorr = r - (r-1)**2/(n-1)

    # 'min(kcorr-1, rcorr-1)': calcula o menor valor entre o número de colunas corrigido menos 1 e o número de linhas corrigido menos 1.
    # 'np.sqrt((chi2corr/n) / (min(kcorr-1, rcorr-1)))': calcula o coeficiente de Cramer's V como a raiz quadrada da razão entre o valor corrigido de qui-quadrado e o menor valor entre o número de colunas e o número de linhas corrigido menos 1.
    return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )


def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display(HTML( '<style>.container { width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\raquel\anaconda3\envs\ds_em_producao\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


## 0.2 Loading Data

In [4]:

# Lê o arquivo todo de um vez
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)

df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# merge, 1º arquivo de referencia, 2º arquivo anexado a referencia, left, 
# coluna que é igual nos dois datasets e serve como chave para eu fazer o merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')



# 1.0 DESCRIÇÃO DOS DADOS

In [5]:

df1 = df_raw.copy()

## 1.1 Rename Columns

In [6]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
            'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
            'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
            'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list(map (snakecase, cols_old))

#rename
df1.columns = cols_new


In [9]:
#mundando o tipo da coluna 'date'
df1['date'] = pd.to_datetime(df1['date'])

## 1.5 Fillout NA

In [10]:
# competition_distance 
# 
# tornar os NA em uma distancia maior que a maior distancia já encontrada no dataset
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)         

# competition_open_since_month    
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

# competition_open_since_year     
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

# promo2_since_week               
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# promo2_since_year               
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)


# promo_interval

#dicionario que associa o mês ao nome do mês
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Ouc', 11: 'Nov', 12: 'Dec',}
 
#preenche com NA para não ter que comparar usando ISNAN, mas poderia ter feito usando ISNAN
df1['promo_interval'].fillna(0, inplace=True)


#extrai o mês da coluna data e aplica o dicionario para fazer a tradução para o nome do mes
df1['month_map'] = df1['date'].dt.month.map(month_map)

# aplica a condicional para saber se o month_map está dentro daquele intervalo para saber se a loja está ou não em promoção
df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

## 1.6 Change Types

In [11]:
df1['competition_open_since_month'] = df1['competition_open_since_month'].astype(int)
df1['competition_open_since_year'] = df1['competition_open_since_year'].astype(int)
df1['promo2_since_week'] = df1['promo2_since_week'].astype(int)
df1['promo2_since_year'] = df1['promo2_since_year'].astype(int)




## 1.7 Descriptive Statistics

In [12]:
num_attributes = df1.select_dtypes(include=['int64', 'float64', 'int32'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'int32', 'datetime64[ns]'])

# 2.0 FEATURE ENGINEERING

In [15]:
df2 = df1.copy()

## 2.4. Feature Engineering

In [16]:
# year
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.isocalendar().week

# year week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W')

# competition since
df2['competition_since'] = df2.apply(lambda x: dt.datetime(year=x['competition_open_since_year'], month=x['competition_open_since_month'], day=1), axis=1 )
df2['competition_time_month'] = ((df2['date'] - df2['competition_since'] )/30).apply(lambda x: x.days).astype(int)

# promo since
df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
df2['promo_since'] = df2['promo_since'].apply(lambda x: dt.datetime.strptime(x + '-1', '%Y-%W-%w') - dt.timedelta(days=7))
df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype(int)

# assortment
df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')

# state holiday
df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day')


# 3.0 FILTRAGEM DE VARIÁVEIS

In [17]:
df3 = df2.copy()

## 3.1 Filtragem das Linhas

In [18]:
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## 3.2 Selecao das Colunas

In [19]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop(cols_drop, axis=1)

# 4.0 ANALISE EXPLORATORIA DOS DADOS

In [20]:
df4 = df3.copy()

# 5.0 DATA PREPARATION

In [21]:
df5 = df4.copy()

## 5.2 Rescaling

###### o rescaling é utilizado para ajustar a escala dos dados para um intervalo específico, como 0 e 1 ou -1 e 1. Isso pode ser útil para modelos que utilizam funções de ativação específicas, como redes neurais.

In [22]:
# Técnica de normalização robusta a outliers, que escalona os dados de forma que a mediana seja igual a 0 e a distribuição interquartil (IQR) seja igual a 1.
rs = RobustScaler()

# Método de rescaling que transforma os dados de forma que eles fiquem no intervalo de 0 a 1.
mms = MinMaxScaler()

# Após aplicar os escalonamentos, o código sobrescreve as variáveis originais no dataframe df5 com os valores escalonados.

# Escalonamento da variável competition distance com RobustScaler
df5['competition_distance'] = rs.fit_transform(df5[['competition_distance']].values)
# Salva o objeto rs (RobustScaler) em um arquivo pickle para uso futuro
pickle.dump(rs, open('parameter/competition_distance_scaler.pkl', 'wb'))

# Escalonamento da variável competition time month com RobustScaler
df5['competition_time_month'] = rs.fit_transform(df5[['competition_time_month']].values)
# Salva o objeto rs (RobustScaler) em um arquivo pickle para uso futuro
pickle.dump(rs, open('parameter/competition_time_month_scaler.pkl', 'wb'))

# Escalonamento da variável promo time week com MinMaxScaler
df5['promo_time_week'] = mms.fit_transform(df5[['promo_time_week']].values)
# Salva o objeto mms (MinMaxScaler) em um arquivo pickle para uso futuro
pickle.dump(mms, open('parameter/promo_time_week_scaler.pkl', 'wb'))

# Escalonamento da variável year com MinMaxScaler
df5['year'] = mms.fit_transform(df5[['year']].values)
# Salva o objeto mms (MinMaxScaler) em um arquivo pickle para uso futuro
pickle.dump(mms, open('parameter/year_scaler.pkl', 'wb'))


## 5.3 Transformação

In [23]:
# state_holiday - One Hot Encoding
df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])

# store_type - Label Encoding
le = LabelEncoder()
df5['store_type'] = le.fit_transform(df5['store_type'])
#pickle.dump(le, open('parameter/store_type_scaler.pkl', 'wb'))

# assortment - Ordinal Encoding
assortment_dict = {'basic': 1, 'extra': 2, 'extended': 3}
df5['assortment'] = df5['assortment'].map(assortment_dict)

#### 5.3.2 Response Variable Transformation

In [24]:
df5['sales'] = np.log1p(df5['sales'])

#### 5.3.3 Nature Transformation

In [25]:
# Adiciona colunas com o seno e cosseno dos dias da semana, meses, dias e semana do ano
# Isso é feito por se tratar de dados cíclicos.

# day of week
df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x: np.sin(x * (2. * np.pi/7)))
df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x: np.cos(x * (2. * np.pi/7)))

# month
df5['month_sin'] = df5['month'].apply(lambda x: np.sin(x * ( 2. * np.pi/12)))
df5['month_cos'] = df5['month'].apply(lambda x: np.cos(x * ( 2. * np.pi/12)))

# day
df5['day_sin'] = df5['day'].apply(lambda x: np.sin(x * (2. * np.pi/30)))
df5['day_cos'] = df5['day'].apply(lambda x: np.cos(x * (2. * np.pi/30)))

# week of year
df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x: np.sin(x * (2. * np.pi/52)))
df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x: np.cos(x * (2. * np.pi/52))) 

# 6.0 FEATURE SELECTION

In [26]:
df6 = df5.copy()

## 6.1 Split dataframe into training and test datase

In [27]:
# lista das colunas que serão removidas
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week' ]

# remove as colunas especificadas
df6 = df6.drop(cols_drop, axis=1)

In [28]:
# seleciona as linhas do DataFrame que possuem datas anteriores a '2015-06-19'
X_train = df6[df6['date'] < '2015-06-19']

# seleciona a coluna 'sales' do DataFrame 'X_train' e armazena em 'y_train'
y_train = X_train['sales']

# seleciona as linhas do DataFrame que possuem datas a partir de '2015-06-19'
X_test = df6[df6['date'] >= '2015-06-19']

# seleciona a coluna 'sales' do DataFrame 'X_test' e armazena em 'y_test'
y_test = X_test['sales']

# exibe a data mínima e máxima do conjunto de treinamento
print('Training Min Date: {}'.format( X_train['date'].min()))
print('Training Max Date: {}'.format( X_train['date'].max()))

# exibe a data mínima e máxima do conjunto de teste
print('\nTest Min Date: {}'.format( X_test['date'].min()))
print('Test Max Date: {}'.format( X_test['date'].max()))

Training Min Date: 2013-01-01 00:00:00
Training Max Date: 2015-06-18 00:00:00

Test Min Date: 2015-06-19 00:00:00
Test Max Date: 2015-07-31 00:00:00


## 6.3 Manual Feature Selection - BORUTA

In [29]:
# Lista com as colunas selecionadas pelo algoritmo Boruta
cols_selected_boruta = [
    'store',
    'promo',
    'store_type',
    'assortment',
    'competition_distance',
    'competition_open_since_month',
    'competition_open_since_year',
    'promo2',
    'promo2_since_week',
    'promo2_since_year',
    'competition_time_month',
    'promo_time_week',
    'day_of_week_sin',
    'day_of_week_cos',
    'month_sin',
    'month_cos',
    'day_sin',
    'day_cos',
    'week_of_year_sin',
    'week_of_year_cos']

# Colunas que precisam ser mantidas no conjunto de dados
feat_to_add = ['date', 'sales']

# Lista completa de colunas selecionadas
cols_selected_boruta_full = cols_selected_boruta.copy()
cols_selected_boruta_full.extend(feat_to_add)


# O resultado final é uma lista cols_selected_boruta_full com os nomes das colunas selecionadas pelo Boruta e as colunas adicionais 'date' e 'sales'.

# 7.0 MACHINE LEARNING MODELLING

In [30]:
# Selecionando as colunas de treinamento e teste que foram selecionadas pelo algoritmo Boruta
x_train = X_train[cols_selected_boruta]
x_test = X_test[cols_selected_boruta]

# Time Series Data Preparation
# Selecionando todas as colunas relevantes, incluindo data e vendas, para treinamento
x_training = X_train[cols_selected_boruta_full]


## 8.2 Final Model

In [31]:
# Define um dicionário com valores fixos para os hiperparâmetros do modelo
param_tuned = {
    'n_estimators': 3000,
    'eta': 0.03,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'min_child_weight': 3
}


In [32]:
# Cria um modelo XGBoostRegressor com os valores especificados em 'param_tuned' e ajusta aos dados de treinamento
model_xgb_tuned = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=param_tuned['n_estimators'],
    eta=param_tuned['eta'],
    max_depth=param_tuned['max_depth'],
    subsample=param_tuned['subsample'],
    colsample_bytree=param_tuned['colsample_bytree'],
    min_child_weight=param_tuned['min_child_weight']
    ).fit(x_train, y_train)

# Faz previsões no conjunto de teste usando o modelo ajustado
yhat_xgb_tuned = model_xgb_tuned.predict(x_test)

# Avalia o desempenho do modelo usando a função 'ml_error'
xgb_result_tuned = ml_error(
    'XGBoost Regressor',
    np.expm1(y_test), # desfaz a transformação logarítmica aplicada nos dados de saída (y)
    np.expm1(yhat_xgb_tuned) # desfaz a transformação logarítmica aplicada nas previsões (yhat)
)

xgb_result_tuned


,Model Name,MAE,MAPE,RMSE
0,XGBoost Regressor,770.209978,0.115623,1108.062869


# 9.0 TRADUCAO E INTERPRETACAO DO ERRO

In [33]:
# Selecionando as colunas relevantes do conjunto de dados de teste e armazenando em df9
df9 = X_test[cols_selected_boruta_full]

# Convertendo as previsões e os valores reais para a escala original de vendas usando np.expm1()
# A coluna "sales" contém os valores reais e "predictions" contém as previsões do modelo
df9['sales'] = np.expm1(df9['sales'])
df9['predictions'] = np.expm1(yhat_xgb_tuned)


C:\Users\raquel\AppData\Local\Temp\ipykernel_11748\2332760266.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df9['sales'] = np.expm1(df9['sales'])
C:\Users\raquel\AppData\Local\Temp\ipykernel_11748\2332760266.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df9['predictions'] = np.expm1(yhat_xgb_tuned)


## 9.1 Business Performance

In [34]:

# Calculando a soma das previsões por loja e armazenando em df91
df91 = df9[['store', 'predictions']].groupby('store').sum().reset_index()

# Calculando o erro absoluto médio (MAE) por loja e armazenando em df9_aux1
df9_aux1 = df9[['store', 'sales', 'predictions']].groupby('store').apply(
    lambda x: mean_absolute_error(x['sales'], x['predictions'])
).reset_index().rename(columns={0: 'MAE'})

# Calculando o erro percentual absoluto médio (MAPE) por loja e armazenando em df9_aux2
df9_aux2 = df9[['store', 'sales', 'predictions']].groupby('store').apply(
    lambda x: mean_absolute_percentage_error(x['sales'], x['predictions'])
).reset_index().rename(columns={0: 'MAPE'})

# Juntando os DataFrames df9_aux1 e df9_aux2 em df9_aux3, usando a coluna 'store' como chave
df9_aux3 = pd.merge(df9_aux1, df9_aux2, how='inner', on='store')

# Juntando os DataFrames df91 e df9_aux3 em df92, usando a coluna 'store' como chave
df92 = pd.merge(df91, df9_aux3, how='inner', on='store')

# Calculando os cenários 'pior caso' e 'melhor caso' usando o MAE, e adicionando como colunas em df92
df92['worst_scenario'] = df92['predictions'] - df92['MAE']
df92['best_scenario'] = df92['predictions'] + df92['MAE']

# Reordenando as colunas de df92
df92 = df92[['store', 'predictions', 'worst_scenario', 'best_scenario', 'MAE', 'MAPE']]


# 10.0 DEPLOY MODEL TO PRODUCTION

In [35]:
# Save Trained Model
pickle.dump(model_xgb_tuned, open(r'C:\Users\raquel\Documents\Comunidade DS\repos\05-DS-emProducao\rossmann_store_sales\model\model_rossmann.pkl', 'wb'))


'''Este código salva o modelo model_xgb_tuned que foi treinado em um
 arquivo com o nome model_rossmann.pkl. O caminho completo para este arquivo 
 é especificado como um argumento para a função open(). O prefixo r antes das 
 aspas duplas indica que esta é uma "string raw" que desativa o processamento 
 de caracteres de escape, para que as barras invertidas no caminho do arquivo 
 não sejam interpretadas como caracteres de escape. A opção wb indica que o 
 arquivo deve ser aberto para escrita em modo binário, o que é necessário para 
 salvar objetos Python em formato binário usando a função pickle.dump().'''


'Este código salva o modelo model_xgb_tuned que foi treinado em um\n arquivo com o nome model_rossmann.pkl. O caminho completo para este arquivo \n é especificado como um argumento para a função open(). O prefixo r antes das \n aspas duplas indica que esta é uma "string raw" que desativa o processamento \n de caracteres de escape, para que as barras invertidas no caminho do arquivo \n não sejam interpretadas como caracteres de escape. A opção wb indica que o \n arquivo deve ser aberto para escrita em modo binário, o que é necessário para \n salvar objetos Python em formato binário usando a função pickle.dump().'

## 10.1 Rossmann Class

In [44]:
import pickle
import inflection
import pandas as pd
import numpy as np
import math
import datetime


class Rossmann(object):
    def __init__(self):
        # Diretório do projeto
        self.home_path = 'C:\\Users\\raquel\\Documents\\Comunidade DS\\repos\\05-DS-emProducao\\rossmann_store_sales\\'
       
        # Carrega os modelos pré-treinados para normalização
        self.competition_distance_scaler = pickle.load(open(self.home_path + 'parameter\\competition_distance_scaler.pkl', 'rb'))
        self.competition_time_month_scaler = pickle.load(open(self.home_path + 'parameter\\competition_time_month_scaler.pkl', 'rb'))
        self.promo_time_week_scaler = pickle.load(open(self.home_path + 'parameter\\promo_time_week_scaler.pkl', 'rb'))
        self.year_scaler = pickle.load(open(self.home_path + 'parameter\\year_scaler.pkl', 'rb'))
        self.store_type_scaler = pickle.load(open(self.home_path + 'parameter\\store_type_scaler.pkl', 'rb'))

    def data_cleaning(self, df1):
        ## 1.1. Renomeando Colunas
        # Lista de colunas antigas
        cols_old = ['Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
                    'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
                    'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
        
        # Transformando o nome das colunas em snake_case
        snakecase = lambda x: inflection.underscore(x)
        cols_new = list(map(snakecase, cols_old))
        
        # renomeando as colunas do dataframe
        df1.columns = cols_new

        ## 1.3. Tipos de Dados
        # Convertendo a coluna "date" para o tipo datetime
        df1['date'] = pd.to_datetime(df1['date'])

        ## 1.5. Preenchimento de Valores Faltantes
        # competition_distance
        # Substituindo valores faltantes por 200000.0
        df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)
        
        # competition_open_since_month
        # Substituindo valores faltantes pelo mês da coluna "date"
        df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)
        
        # competition_open_since_year
        # Substituindo valores faltantes pelo ano da coluna "date"
        df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)
        
        # promo2_since_week
        # Substituindo valores faltantes pelo número da semana da coluna "date"
        df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)
        
        # promo2_since_year
        # Substitui os valores faltantes da coluna "promo2_since_year" pelo ano presente na coluna "date" caso seja possível
        df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

        # promo_interval
        # Mapeamento dos meses do ano para as siglas correspondentes
        month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun',
                    7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

        # Substitui os valores faltantes na coluna "promo_interval" por 0
        df1['promo_interval'].fillna(0, inplace=True )

        # Cria uma nova coluna "month_map" que mapeia o número do mês para as siglas correspondentes
        df1['month_map'] = df1['date'].dt.month.map( month_map )

        # Cria uma nova coluna "is_promo" que informa se determinada loja esteve em promoção ou não no mês da data presente na linha
        df1['is_promo'] = df1[['promo_interval', 'month_map']].apply( lambda x:
            0 if x['promo_interval'] == 0 else 1 if x['month_map'] in
            x['promo_interval'].split( ',' ) else 0, axis=1 )
            

        ## 1.6. Change Data Types
        # competiton
        # Converte a coluna "competition_open_since_month" para tipo inteiro
        df1['competition_open_since_month'] = df1['competition_open_since_month'].astype( int )

        # Converte a coluna "competition_open_since_year" para tipo inteiro
        df1['competition_open_since_year'] = df1['competition_open_since_year'].astype( int )

        # promo2
        # Converte a coluna "promo2_since_week" para tipo inteiro
        df1['promo2_since_week'] = df1['promo2_since_week'].astype( int )

        # Converte a coluna "promo2_since_year" para tipo inteiro
        df1['promo2_since_year'] = df1['promo2_since_year'].astype( int )

        # Retorna o dataframe com as alterações realizadas
        return df1
    

    def feature_engineering(self, df2):
        # year
        df2['year'] = df2['date'].dt.year
        
        # month
        df2['month'] = df2['date'].dt.month
        
        # day
        df2['day'] = df2['date'].dt.day
        
        # week of year
        df2['week_of_year'] = df2['date'].dt.weekofyear
        
        # year week
        df2['year_week'] = df2['date'].dt.strftime('%Y-%W')
        
        # competition since
        df2['competition_since'] = df2.apply(lambda x: datetime.datetime(
            year=x['competition_open_since_year'],
            month=x['competition_open_since_month'],
            day=1
        ), axis=1)
        df2['competition_time_month'] = ((df2['date'] - df2['competition_since']) / 30).apply(lambda x: x.days).astype(int)
        
        # promo since
        df2['promo_since'] = df2['promo2_since_year'].astype(str) + '-' + df2['promo2_since_week'].astype(str)
        df2['promo_since'] = df2['promo_since'].apply(lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.timedelta(days=7))
        df2['promo_time_week'] = ((df2['date'] - df2['promo_since']) / 7).apply(lambda x: x.days).astype(int)
        
        # assortment
        df2['assortment'] = df2['assortment'].apply(lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended')
        
        # state holiday
        df2['state_holiday'] = df2['state_holiday'].apply(lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day')
        
        # 3.0 - FILTRAGEM DE VARIÁVEIS
        ## 3.1. Filtragem das Linhas
        df2 = df2[df2['open'] != 0]
        
        ## 3.2. Selecao das Colunas
        cols_drop = ['open', 'promo_interval', 'month_map']
        df2 = df2.drop(cols_drop, axis=1)
        
        return df2
    

    def data_preparation(self, df5):
        ## 5.2. Rescaling

        # redimensionamento das variáveis numéricas para deixá-las na mesma escala
        # distância até a concorrência
        df5['competition_distance'] = self.competition_distance_scaler.fit_transform(df5[['competition_distance']].values)
        
        # tempo em meses desde a última competição
        df5['competition_time_month'] = self.competition_time_month_scaler.fit_transform(df5[['competition_time_month']].values)
       
        # tempo em semanas desde a última promoção
        df5['promo_time_week'] = self.promo_time_week_scaler.fit_transform(df5[['promo_time_week']].values)
       
        # ano
        df5['year'] = self.year_scaler.fit_transform(df5[['year']].values)


        ### 5.3.1. Encoding
        # state_holiday - One Hot Encoding
        # transformação de variáveis categóricas em variáveis binárias
        df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns=['state_holiday'])
        
        # store_type - Label Encoding
        # transformação de variáveis categóricas em valores numéricos
        df5['store_type'] = self.store_type_scaler.fit_transform(df5['store_type'])
       
        # assortment - Ordinal Encoding
        # transformação de variáveis categóricas em valores ordinais
        assortment_dict = {'basic': 1, 'extra': 2, 'extended': 3}
        df5['assortment'] = df5['assortment'].map(assortment_dict)
        
        ### 5.3.3. Nature Transformation
        # day of week
        # transformação de variáveis de tempo em valores numéricos utilizando funções trigonométricas
        df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x: np.sin(x * (2. * np.pi/7)))
        df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x: np.cos(x * (2. * np.pi/7)))
        
        # month
        df5['month_sin'] = df5['month'].apply(lambda x: np.sin(x * (2. * np.pi/12)))
        df5['month_cos'] = df5['month'].apply(lambda x: np.cos(x * (2. * np.pi/12)))
        
        # day
        df5['day_sin'] = df5['day'].apply(lambda x: np.sin(x * (2. * np.pi/30)))
        df5['day_cos'] = df5['day'].apply(lambda x: np.cos(x * (2. * np.pi/30)))

        # week of year
        df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x: np.sin(x * (2. * np.pi/52)))
        df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x: np.cos(x * (2. * np.pi/52)))

        cols_selected = ['store', 'promo', 'store_type', 'assortment', 'competition_distance', 'competition_open_since_month',
                         'competition_open_since_year', 'promo2', 'promo2_since_week', 'promo2_since_year', 
                         'competition_time_month', 'promo_time_week', 'day_of_week_sin', 'day_of_week_cos', 'month_sin', 
                         'month_cos', 'day_sin', 'day_cos', 'week_of_year_sin', 'week_of_year_cos']
                 
        return df5[cols_selected]


def get_prediction(self, model, original_data, test_data):
    
    # prediction
    pred = model.predict(test_data)
    
    # join pred into the original data
    original_data['prediction'] = np.expm1(pred)
    
    return original_data.to_json(orient='records', date_format='iso')


## 10.2 API Handler

In [37]:
import pickle
import pandas as pd
from flask import Flask, request, Response
from rossmann.Rossmann import Rossmann


# Carregando o modelo salvo
model = pickle.load(open('C:\\Users\\raquel\\Documents\\Comunidade DS\\repos\\05-DS-emProducao\\rossmann_store_sales\\model\\model_rossmann.pkl', 'rb'))



# Inicializando a API
app = Flask(__name__)

# Definindo a rota e o método POST para a função de predição
@app.route('/rossmann/predict', methods=['POST'])
def rossmann_predict():
    # Obtendo os dados em formato JSON
    test_json = request.get_json()

    # Verificando se há dados presentes
    if test_json:
        # Se houver apenas um exemplo
        if isinstance(test_json, dict):
            test_raw = pd.DataFrame(test_json, index=[0])
        # Se houverem múltiplos exemplos
        else:
            test_raw = pd.DataFrame(test_json, columns=test_json[0].keys())

        # Instanciando a classe Rossmann
        pipeline = Rossmann()

        # Realizando a limpeza dos dados
        df1 = pipeline.data_cleaning(test_raw)

        # Realizando a engenharia de features
        df2 = pipeline.feature_engineering(df1)

        # Preparando os dados
        df3 = pipeline.data_preparation(df2)

        # Realizando a predição
        df_response = pipeline.get_prediction(model, test_raw, df3)

        # Retornando os resultados em formato JSON
        return df_response

    else:
        # Caso não hajam dados presentes, retornando um JSON vazio com código 200
        return Response('{}', status=200, mimetype='application/json')

# Iniciando a aplicação
if __name__ == '__main__':
    app.run('0.0.0.0')


ModuleNotFoundError: No module named 'rossmann'

## 10.3 API Tester

In [49]:
# loading test dataset
df10 = pd.read_csv( 'C:\\Users\\raquel\\Documents\\Comunidade DS\\repos\\05-DS-emProducao\\rossmann_store_sales\\data\\test.csv')

In [76]:
# merge test dataset + store
df_test = pd.merge( df10, df_store_raw, how='left', on='Store' )
# choose store for prediction
df_test = df_test[df_test['Store'].isin( [15, 24, 39] )]
# remove closed days
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()]
df_test = df_test.drop( 'Id', axis=1 )

In [77]:
import json

# convert Dataframe to json
data = json.dumps( df_test.to_dict( orient='records' ) )

In [78]:
import requests

# API Call
url = 'https://rossmann-model-api.onrender.com/rossmann/predict'
#url = 'http://127.0.0.1:5000/rossmann/predict'
header = {'Content-type': 'application/json' }
data = data
r = requests.post(url, data=data, headers=header)
print('Status Code {}'.format( r.status_code))

Status Code 200


In [79]:
d1 = pd.DataFrame(r.json(), columns=r.json()[0].keys())

In [80]:
d2 = d1[['store', 'prediction']].groupby( 'store' ).sum().reset_index()

for i in range( len( d2 ) ):
    print('Store Number {} will sell R${:,.2f} in the next 6 weeks'.format(
        d2.loc[i, 'store'],
        d2.loc[i, 'prediction']))

Store Number 15 will sell R$303,752.68 in the next 6 weeks
Store Number 24 will sell R$354,434.94 in the next 6 weeks
Store Number 39 will sell R$155,208.82 in the next 6 weeks
